# compute accuracy in 5 downstream tasks

after getting classification in 5 tasks, compute the accuracy between labels and logits.


In [1]:
import pandas as pd
import re

import os
from together import Together

from openai import OpenAI
import random
from collections import Counter
import time

## acc of each category in each downstream task

In [93]:
# model_name='gpt-4o'
# model_name='llama3.1-70b-instruct'
# model_name = 'llama3.1-8b-instruct'
# model_name = 'gemma2-9b-instruct'
# model_name= 'mistral-7b-instruct'
model_name = 'qwen2-72b-instruct'

# model_name = 'sentence'
# model_name = 'random_sentence'

judge_model_name = "gpt-4o-mini"
# judge_model_name = "gpt-4o-turbo"

print(model_name)
print(judge_model_name)

qwen2-72b-instruct
gpt-4o-mini


In [94]:

augmentation_method = 'prompt_aug_sentence_cls'
# augmentation_method = 'conditional_recommend'
# augmentation_method = 'fewshot'
augmentation_bool=True
print(augmentation_bool)
print(augmentation_method)

print(model_name)

True
prompt_aug_sentence_cls
qwen2-72b-instruct


In [97]:
!ls ./dataset_with_logits/{model_name}/prompt_aug_sentence_cls/stance/dataset_logits_qwen2-72b-instruct_prompt_aug_sentence_cls_stance_5039.csv

# !ls ./dataset_finalOutput/gpt-4o/dataset_finalOutput_gpt-4o_gpt-4o-mini_prompt_aug_sentence_cls_5039.csv


./dataset_with_logits/qwen2-72b-instruct/prompt_aug_sentence_cls/stance/dataset_logits_qwen2-72b-instruct_prompt_aug_sentence_cls_stance_5039.csv


In [98]:


if model_name == 'sentence' or model_name == 'random_sentence': 
    dataset_path = f'./dataset_with_logits/{model_name}/stance/dataset_labels_using_{model_name}_stance_5039.csv'
    print(dataset_path)
    emoji_dataset = pd.read_csv(dataset_path)
else: 
    if augmentation_bool == True: 
        dataset_path = f'./dataset_with_logits/{model_name}/prompt_aug_sentence_cls/stance/dataset_logits_{model_name}_prompt_aug_sentence_cls_stance_5039.csv'
        print(dataset_path)
        emoji_dataset = pd.read_csv(dataset_path)
    else:
        dataset_path = f'./dataset_with_logits/{model_name}/stance/dataset_logits_{model_name}_stance_5039.csv'
        print(dataset_path)
        emoji_dataset = pd.read_csv(dataset_path)


emoji_dataset.info()

./dataset_with_logits/qwen2-72b-instruct/prompt_aug_sentence_cls/stance/dataset_logits_qwen2-72b-instruct_prompt_aug_sentence_cls_stance_5039.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 38 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Unnamed: 0.1                                     5039 non-null   int64 
 1   Unnamed: 0                                       5039 non-null   int64 
 2   index                                            5039 non-null   int64 
 3   user_id                                          5039 non-null   object
 4   emoji_sentence                                   5039 non-null   object
 5   char_count                                       5039 non-null   int64 
 6   cluster                                          5039 non-null   int64 
 7   sentence                                         

In [86]:
# !ls ./condition_recommed_fewshot/dataset_logits_gemma2-9b-instruct_conditional_recommend_stance_5039.csv

!ls ./dataset_with_logits/mistral-7b-instruct/prompt_aug_sentence_cls/emotion/dataset_logits_mistral-7b-instruct_prompt_aug_sentence_cls_emotion_2400.csv


./dataset_with_logits/mistral-7b-instruct/prompt_aug_sentence_cls/emotion/dataset_logits_mistral-7b-instruct_prompt_aug_sentence_cls_emotion_2400.csv


In [88]:
dataset_path = './condition_recommed_fewshot/dataset_logits_gemma2-9b-instruct_conditional_recommend_stance_5039.csv'
dataset_path = './dataset_with_logits/mistral-7b-instruct/prompt_aug_sentence_cls/emotion/dataset_logits_mistral-7b-instruct_prompt_aug_sentence_cls_emotion_2400.csv'

In [89]:
emoji_dataset=pd.read_csv(dataset_path)

dataset_len=len(emoji_dataset)
print(dataset_len)

5039


In [90]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 38 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Unnamed: 0.1                                     5039 non-null   int64 
 1   Unnamed: 0                                       5039 non-null   int64 
 2   index                                            5039 non-null   int64 
 3   user_id                                          5039 non-null   object
 4   emoji_sentence                                   5039 non-null   object
 5   char_count                                       5039 non-null   int64 
 6   cluster                                          5039 non-null   int64 
 7   sentence                                         5039 non-null   object
 8   gender_labels                                    5039 non-null   object
 9   sentiment_labels                         

In [99]:
# gender

task_list = ['gender', 'sentiment', 'emotion', 'age', 'stance']

category_dict = {
    'gender': ['male', 'female'],
    'sentiment': ['positive', 'negative', 'neutral'],
    'emotion': ['happiness', 'sadness', 'fear', 'anger', 'surprise', 'disgust'],
    'age': ['child', 'teen', 'adult', 'senior'],
    'stance': ['favor', 'none', 'against']
}




In [100]:

augmentation_method = 'prompt_aug_sentence_cls'
# augmentation_method = 'conditional_recommend'
# augmentation_method = 'fewshot'
augmentation_bool=True
print(augmentation_bool)
print(augmentation_method)

True
prompt_aug_sentence_cls


In [101]:

acc_count_total=0

for task in task_list: 
    print(f'the task is: {task}')
    column_name_labels = f'{task}_labels'
    if model_name =='sentence' or model_name == 'random_sentence': 
        column_name_logits = f'{task}_labels_using_{model_name}'
    else: 
        if augmentation_bool == True: 
            column_name_logits = f'{task}_logits_{augmentation_method}'
        else:
            column_name_logits = f'{task}_logits'

        
    temp_category_list = category_dict[task]
    acc_dict = {}
    for category in temp_category_list: 
        df = emoji_dataset[emoji_dataset[column_name_labels] == category]
        cnt_correct = (df[column_name_logits] == df[column_name_labels]).sum()
        cnt_total = len(df)
        accuracy = cnt_correct / cnt_total if cnt_total > 0 else 0

        acc_dict[category] = float(accuracy)
    print(acc_dict)

    matches = emoji_dataset[column_name_labels] == emoji_dataset[column_name_logits]
    # Count the number of True values in the boolean Series, which indicates matches
    num_matches = matches.sum()
    print(f"Number of same elements in {task}: {num_matches}")

    acc_count_total+=int(num_matches)
    acc_dict['overall']=int(num_matches)/dataset_len

    if augmentation_bool==True:
        folder_path = f'./acc_output/{model_name}/{augmentation_method}/{task}'
        output_filename = f"./acc_output/{model_name}/{augmentation_method}/{task}/acc_{model_name}_{task}_{augmentation_method}_{dataset_len}.txt"
    else: 
        folder_path = f"./acc_output/{model_name}/{task}"
        output_filename = f"./acc_output/{model_name}/{task}/acc_{model_name}_{task}_{dataset_len}.txt"
    os.makedirs(folder_path, exist_ok=True)

    # # Save the results to a text file named after the model
    # output_filename = f"./acc_output/{model_name}/{task}/acc_{model_name}_{task}_{dataset_len}.txt"
    
    with open(output_filename, "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f'augmentation_method: {augmentation_method}\n')
        f.write(f'Task: {task}\n')
        for key, value in acc_dict.items():
            f.write(f'Accuracy for {key}: {value}\n')
    print(f"acc saved in {output_filename}")

acc_avg=acc_count_total/(5*dataset_len)
# # Save the results to a text file named after the model
# output_filename = f"./acc_output/{model_name}/acc_avg_{model_name}_{dataset_len}.txt"

if augmentation_bool==True:
    output_filename = f"./acc_output/{model_name}/{augmentation_method}/acc_{model_name}_{augmentation_method}_{dataset_len}.txt"
else: 
    output_filename = f"./acc_output/{model_name}/acc_{model_name}_{dataset_len}.txt"

with open(output_filename, "w") as f:
    f.write(f"Model: {model_name}\n")
    f.write(f'augmentation_method: {augmentation_method}\n')
    f.write(f'Accuracy Avg: {acc_avg}\n')
print(f"acc saved in {output_filename}")



the task is: gender
{'male': 0.6706204379562044, 'female': 0.9097295398665262}
Number of same elements in gender: 4060
acc saved in ./acc_output/qwen2-72b-instruct/prompt_aug_sentence_cls/gender/acc_qwen2-72b-instruct_gender_prompt_aug_sentence_cls_5039.txt
the task is: sentiment
{'positive': 0.8565153733528551, 'negative': 0.891219512195122, 'neutral': 0.3648936170212766}
Number of same elements in sentiment: 3925
acc saved in ./acc_output/qwen2-72b-instruct/prompt_aug_sentence_cls/sentiment/acc_qwen2-72b-instruct_sentiment_prompt_aug_sentence_cls_5039.txt
the task is: emotion
{'happiness': 0.7743174061433447, 'sadness': 0.6741573033707865, 'fear': 0.5586206896551724, 'anger': 0.7710982658959538, 'surprise': 0.49070631970260226, 'disgust': 0.16393442622950818}
Number of same elements in emotion: 3166
acc saved in ./acc_output/qwen2-72b-instruct/prompt_aug_sentence_cls/emotion/acc_qwen2-72b-instruct_emotion_prompt_aug_sentence_cls_5039.txt
the task is: age
{'child': 0.7560503388189739,